## **INSTALLING PACKAGES**

In [1]:
!pip install "ibm-watsonx-ai==0.2.6"
!pip install "langchain ==0.1.16"
!pip install "langchain-ibm ==0.1.4"
!pip install "huggingface ==0.0.1"
!pip install "huggingface-hub ==0.21.4"
!pip install "sentence transformers ==2.2.2"
!pip install "chromadb ==0.4.24"
!pip install "wget==3.2"
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
diffusers 0.31.0 requires huggingface-hub>=0.23.2, but you have huggingface-hub 0.21.4 which is incompatible.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.21.4 which is incompatible.
transformers 4.47.1 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.21.4 which is incompatible.
ERROR: Invalid requirement: 'sentence transformers ==2.2.2': Expected end or semicolon (after name and no valid version specifier)
    sentence transformers ==2.2.2
             ^
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


## **IMPORT LIBRARIES**


In [2]:

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
import wget


In [3]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## **Loading TXT file**


In [4]:
def warn(*args, **kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings('ignore')


loader = TextLoader("/content/Will Doc.txt")
documents = loader.load()


In [5]:

with open('/content/Will Doc.txt', 'r') as f:
    file_contents = f.read()
file_contents

'I, Rajesh Kumar, aged 58 years, S/O Late Shyam Lal, resident of House No. 1234, Sector 45, District Gurgaon, State Haryana, do hereby make and declare this WILL, in a sound and disposing state of mind, with my own free will and consent, without any undue pressure or influence of anybody and with my full senses as under:\n\nThat at present, I am residing with my wife Mrs. Sunita Kumari, both my sons namely Mr. Amit Kumar and Mr. Sumit Kumar, along with their families at the above said address.\n\nI am executing this WILL with the motive that the disputes and differences may not arise among my legal heirs after my death in respect of my assets and properties.\n\nI state that I am the sole owner in possession of House No. 1234, Sector 45, District Gurgaon, State Haryana. Besides this, I am also having two saving bank accounts in State Bank of India, Sector 50, District Gurgaon, State Haryana, and HDFC Bank, Sector 42, District Gurgaon, State Haryana, and a locker with HDFC Bank, Sector 4

# **Splitting the TXT file**

In [6]:
loader = TextLoader("/content/Will Doc.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
nodes = text_splitter.split_documents(documents)
print(len(nodes))

7


# **Creating and storing vector space embeddings**


In [7]:


embeddings = HuggingFaceEmbeddings()
db = Chroma.from_documents(documents , embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Generating LLM Models**


### **LLAMA model(1B)**

In [8]:
llm = Groq(model="llama-3.2-1b-preview", api_key='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')

In [9]:
embed_model=HuggingFaceEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from llama_index.core import Document as LlamaDocument


llama_documents = [
    LlamaDocument(text=doc.page_content, metadata=doc.metadata)
    for doc in documents
]

# Now use llama_documents when creating the index
vector_index = VectorStoreIndex.from_documents(
    documents=llama_documents,  # Use LlamaIndex documents
    show_progress=True,
    service_context=service_context,
    node_parser=SentenceSplitter()
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

In [12]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

In [13]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [14]:
query_engine = index.as_query_engine(service_context=service_context)

In [15]:
query = "What are assets?"
resp = query_engine.query(query)

print(resp.response)

Assets are the movable and immovable properties, money, and other valuable items that the testator (Rajesh Kumar) wants to be inherited by his heirs after his death.


In [16]:
query = "Who is an Executor?"
resp = query_engine.query(query)

print(resp.response)

An Executor is a person appointed by the testator (the person making the Will) to carry out the terms of the Will, specifically to manage and administer the estate and trusts created by the Will.


## **LLAMA model(3B)**

In [17]:
llm = Groq(model="llama-3.2-3b-preview", api_key='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')

In [18]:
embed_model=HuggingFaceEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [19]:
# Create the vector index using the LlamaIndex documents
vector_index = VectorStoreIndex.from_documents(
    llama_documents,
    show_progress=True,
    service_context=service_context,
    node_parser=SentenceSplitter()
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

In [21]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

In [22]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [23]:
query_engine = index.as_query_engine(service_context=service_context)

In [24]:
query = "What are assets?"
resp = query_engine.query(query)

print(resp.response)

Assets are property or resources that have economic value and can be used to generate income or be sold for financial gain. In the context of the provided will document, assets include the house, two saving bank accounts, and a locker, which are mentioned as being inherited by the testator's wife and sons.


In [25]:
query = "Who is an Executor?"
resp = query_engine.query(query)

print(resp.response)

An Executor is a person appointed by the testator (the person making the Will) to carry out the terms of the Will, manage the estate, and make decisions regarding the distribution of assets.


## **LLAMA model(8B)**

In [26]:
llm = Groq(model="llama-3.1-8b-instant", api_key='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')

In [27]:
embed_model=HuggingFaceEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [28]:
# Create the vector index using the LlamaIndex documents
vector_index = VectorStoreIndex.from_documents(
    llama_documents,
    show_progress=True,
    service_context=service_context,
    node_parser=SentenceSplitter()
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [30]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

In [31]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

In [32]:
query_engine = index.as_query_engine(service_context=service_context)

In [33]:
query = "What are assets?"
resp = query_engine.query(query)

print(resp.response)

Assets refer to items of value that an individual or organization owns or controls, such as properties, bank accounts, and other valuables. In the context of a person's estate, assets can include real estate, cash, investments, and personal belongings.


In [34]:
query = "Who is an Executor?"
resp = query_engine.query(query)

print(resp.response)

An Executor is an individual appointed by a person to manage and carry out the instructions outlined in their Will after their death. They are responsible for ensuring that the deceased person's wishes are fulfilled and that their estate is distributed according to their Will.


## **LLAMA model(70B)**

In [35]:
llm = Groq(model="llama-3.3-70b-versatile", api_key='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')

In [36]:
embed_model=HuggingFaceEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [37]:
# Create the vector index using the LlamaIndex documents
vector_index = VectorStoreIndex.from_documents(
    llama_documents,
    show_progress=True,
    service_context=service_context,
    node_parser=SentenceSplitter()
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

In [39]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

In [40]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [41]:
query_engine = index.as_query_engine(service_context=service_context)

In [42]:
query = "What are assets?"
resp = query_engine.query(query)

print(resp.response)

Assets include House No. 1234, Sector 45, District Gurgaon, State Haryana, two saving bank accounts in State Bank of India and HDFC Bank, and a locker with HDFC Bank. Additionally, any movable or immovable properties that may be acquired in the future are also considered assets.


In [43]:
query = "Who is an Executor?"
resp = query_engine.query(query)

print(resp.response)

The Executor is Mr./Mrs. Ravi Sharma, residing at House No. 567, Sector 55, Gurgaon, or alternatively, Mr./Mrs. Priya Singh, residing at House No. 789, Sector 60, Gurgaon, if Mr./Mrs. Ravi Sharma predeceases, is unable, or unwilling to act as the Executor.


## **Gemma model(9B)**

In [44]:
llm = Groq(model="gemma2-9b-it", api_key='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')

In [45]:
embed_model=HuggingFaceEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [46]:
# Create the vector index using the LlamaIndex documents
vector_index = VectorStoreIndex.from_documents(
    llama_documents,
    show_progress=True,
    service_context=service_context,
    node_parser=SentenceSplitter()
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [47]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

In [48]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

In [49]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [50]:
query_engine = index.as_query_engine(service_context=service_context)

In [51]:
query = "What are the assets?"
resp = query_engine.query(query)

print(resp.response)

A house, two bank accounts, and a locker.  



In [52]:
query = "Who is an Executor?"
resp = query_engine.query(query)

print(resp.response)

Mr./Mrs. Ravi Sharma, R/o House No. 567, Sector 55, Gurgaon, is appointed as the sole executor of the Will. If he/she predeceases the testator or is unable or unwilling to act, then Mr./Mrs. Priya Singh, R/o House No. 789, Sector 60, Gurgaon, shall be appointed. 



******************************

### **Risk assessment in legal clausess**

In [95]:
import os
import re
import json
from transformers import pipeline

# Optimize chunk size to reduce processing time
CHUNK_SIZE = 450

def load_and_preprocess(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File '{file_path}' does not exist.")

    # Load the document
    with open(file_path, 'r', encoding='utf-8') as file:
        document = file.read()

    # Split the document into chunks of CHUNK_SIZE characters
    chunks = [document[i:i+CHUNK_SIZE] for i in range(0, len(document), CHUNK_SIZE)]
    return chunks

def risk_detection(chunks, batch_size=8):
    model_name = "google/flan-t5-base"

    nlp = pipeline("text2text-generation", model=model_name)

    results = [] # A single list to store both risk and recommendation for each chunk
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        risk_prompts = [
            (
                "Rewrite the following legal text in plain language suitable for non-legal professionals, without losing critical details:\n\n" + chunk


            )
            for chunk in batch
        ]
        recommendation_prompts=[
            (
               "Highlight clauses in the given contract that could be negotiable or problematic. Suggest revisions or alternative wording to make the terms more favorable:\n\n" + chunk
            )
            for chunk in batch
        ]

        # Generate responses for the batch
        batch_risk_results = nlp(risk_prompts, max_length=400, do_sample=False)
        batch_recommendation_results = nlp(recommendation_prompts, max_length=400, do_sample=False)

        # Combine risk and recommendation for each chunk into a single dictionary
        for chunk, risk_result, recommendation_result in zip(batch, batch_risk_results, batch_recommendation_results):
            results.append({
                "context": chunk,
                "risk_analysis": risk_result['generated_text'],
                "recommendation": recommendation_result['generated_text']
            })

    return results

def main(file_path):
    print("Loading and preprocessing document...")
    chunks = load_and_preprocess(file_path)

    print("Detecting risks and generating recommendations...")
    analysis = risk_detection(chunks)

    # Save results to a JSON file
    output_path = "risk_analysis.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(analysis, f, ensure_ascii=False, indent=4)
    print(f"Analysis complete. Results saved to {output_path}")

if __name__ == "__main__":
    file_path = "Will Doc.txt"
    main(file_path)

Loading and preprocessing document...
Detecting risks and generating recommendations...


KeyboardInterrupt: 

In [93]:
import json
# Load data from JSON file
with open("risk_analysis.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Iterate through the data (which is now a list of dictionaries)
for entry in data:
    print("Context:\n", entry["context"])
    print("Risk Analysis:\n", entry["risk_analysis"])
    print("Recommendation:\n", entry["recommendation"])  # Access the recommendation
    print("-" * 80)


Context:
 I, Rajesh Kumar, aged 58 years, S/O Late Shyam Lal, resident of House No. 1234, Sector 45, District Gurgaon, State Haryana, do hereby make and declare this WILL, in a sound and disposing state of mind, with my own free will and consent, without any undue pressure or influence of anybody and with my full senses as under:

That at present, I am residing with my wife Mrs. Sunita Kumari, both my sons namely Mr. Amit Kumar and Mr. Sumit Kumar, along with their families at the above said address.

I a
Risk Analysis:
 (ii)
Recommendation:
 hereby declare this WILL, in a sound and disposing state of mind, with my own free will and consent, without any undue pressure or influence of anybody and with my full senses as under: That at present, I am residing with my wife Mrs. Sunita Kumari, both my sons namely Mr. Amit Kumar and Mr. Sumit Kumar, along with their families at the above said address.
--------------------------------------------------------------------------------
Context:
 m

In [86]:
import pandas as pd

# Load data from JSON file
with open("risk_analysis.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert data to a Pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df

,context,risk_analysis,recommendation
0,"I, Rajesh Kumar, aged 58 years, S/O Late Shyam...",(ii),"hereby declare this WILL, in a sound and dispo..."
1,m executing this WILL with the motive that the...,The following are the assets and properties of...,I have a single heir in possession of House No...
2,"rict Gurgaon, State Haryana.\n\nThat by virtue...",The text is a list of possible risks and depen...,"is to be disposed of in the name of my wife, M..."
3,", District Gurgaon, State Haryana, shall devol...",My sons shall become the absolute owners of Ho...,my sons shall be entitled to the money and art...
4,my sons namely Mr. Amit Kumar and Mr. Sumit K...,The text is a list of the following:,appointment of a trustee
5,"f an Executor\nI appoint Mr./Mrs. Ravi Sharma,...",The text is intended to be a guide only and do...,Priya Singh
6,he contrary.\n\nPowers of my Executor\nTo carr...,he contrary.,a mortgage on the property.
7,rty will take the property subject to that mor...,To take all legal actions to have the probate ...,a result of the probate proceeding;
8,"specified in this will, and if not specifical...","a) to invest, manage, lease, rent, exchange, m...",remuneration
9,ion concerning any investments or securities w...,Executor,Executor
